In [1]:
pip install pandas openpyxl shapely


   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 16.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
from shapely.geometry import Point, Polygon

# 1. Coordenadas del polígono del Distrito 15
coordenadas_distrito_15 = [
    (-65.228634, -26.862296), (-65.2274581, -26.8623902), (-65.2259817, -26.8624844),
    (-65.2245096, -26.8627495), (-65.223325, -26.8629924), (-65.2225409, -26.8631316),
    (-65.2219596, -26.8633024), (-65.2211566, -26.8633545), (-65.2207144, -26.8635259),
    (-65.220214, -26.8635367), (-65.2208827, -26.8654954), (-65.221138, -26.866482),
    (-65.2191161, -26.8668016), (-65.216543, -26.867214), (-65.2147802, -26.8675886),
    (-65.213709, -26.867835), (-65.2127599, -26.8680207), (-65.210833, -26.868323),
    (-65.2090275, -26.8656818), (-65.2074811, -26.8633084), (-65.2069646, -26.8634617),
    (-65.2056128, -26.8615667), (-65.2035821, -26.8608005), (-65.2029677, -26.8586943),
    (-65.2026011, -26.8571523), (-65.2022178, -26.8559527), (-65.201579, -26.853698),
    (-65.200628, -26.849737), (-65.199678, -26.84576), (-65.199603, -26.845451),
    (-65.199584, -26.845297), (-65.200651, -26.845048), (-65.200601, -26.844596),
    (-65.218482, -26.84118), (-65.222168, -26.855252), (-65.22301, -26.856942),
    (-65.223736, -26.857607), (-65.2243197, -26.8580847), (-65.2262885, -26.8598026),
    (-65.2275089, -26.8607668), (-65.2280119, -26.8615169), (-65.2283706, -26.8619302),
    (-65.228634, -26.862296)
]
poligono = Polygon(coordenadas_distrito_15)

# 2. Leer el archivo
df = pd.read_excel("datos_originales.xlsx")

# 3. Limpiar nombres de columnas
df.columns = df.columns.str.strip()

# 4. Forzar conversión de lat/lon a float (coercing errores como "Error" a NaN)
df["Latitud"] = pd.to_numeric(df["Latitud"], errors="coerce")
df["Longitud"] = pd.to_numeric(df["Longitud"], errors="coerce")

# 5. Eliminar filas con lat/lon inválidos
df = df.dropna(subset=["Latitud", "Longitud"])

# 6. Verificar si cada punto está dentro del polígono
df["Dentro Distrito 15"] = df.apply(
    lambda row: poligono.contains(Point(row["Longitud"], row["Latitud"])),
    axis=1
)

# 7. Filtrar puntos que están dentro
df_distrito15 = df[df["Dentro Distrito 15"]].drop(columns=["Dentro Distrito 15"])

# 8. Exportar resultados
df_distrito15.to_excel("puntos_dentro_distrito15.xlsx", index=False)

print(f"✅ Se exportaron {len(df_distrito15)} puntos dentro del Distrito 15.")



✅ Se exportaron 285 puntos dentro del Distrito 15.


In [3]:
import pandas as pd

df = pd.read_excel("datos_originales.xlsx")
print(df.columns.tolist())


['Fecha', 'Direccion', 'Procedimiento Destacado', 'Latitud ', 'Longitud']


In [6]:
pip install geopy



   ---------------------------------------- 0/2 [geographiclib]
   -------------------- ------------------- 1/2 [geopy]
   -------------------- ------------------- 1/2 [geopy]
   -------------------- ------------------- 1/2 [geopy]
   -------------------- ------------------- 1/2 [geopy]
   -------------------- ------------------- 1/2 [geopy]
   -------------------- ------------------- 1/2 [geopy]
   ---------------------------------------- 2/2 [geopy]

Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
from geopy.distance import geodesic

# 1. Cargar el archivo filtrado previamente
archivo = "puntos_dentro_distrito15.xlsx"
df = pd.read_excel(archivo)

# 2. Asegurar que no haya espacios en los nombres de columna
df.columns = df.columns.str.strip()

# 3. Separar los relevamientos y los otros
df_relevamiento = df[df["Procedimiento Destacado"].str.contains("Relevamiento", case=False, na=False)].copy()
df_otros = df[~df.index.isin(df_relevamiento.index)].copy()

# 4. Inicializar columna para marcar agrupados
df_relevamiento["agrupado"] = False
agrupados = []

# 5. Definir la tolerancia (en metros) para puntos cercanos
tolerancia_metros = 15

# 6. Agrupar puntos cercanos entre sí
for idx, row in df_relevamiento.iterrows():
    if df_relevamiento.at[idx, "agrupado"]:
        continue

    punto_base = (row["Latitud"], row["Longitud"])
    grupo = [idx]
    df_relevamiento.at[idx, "agrupado"] = True

    for jdx, row2 in df_relevamiento.iterrows():
        if idx == jdx or df_relevamiento.at[jdx, "agrupado"]:
            continue

        punto_comp = (row2["Latitud"], row2["Longitud"])
        distancia = geodesic(punto_base, punto_comp).meters

        if distancia <= tolerancia_metros:
            grupo.append(jdx)
            df_relevamiento.at[jdx, "agrupado"] = True

    fila_resultado = df_relevamiento.loc[idx].copy()
    fila_resultado["Cantidad afectada"] = len(grupo)
    agrupados.append(fila_resultado)

# 7. Crear DataFrame con relevamientos agrupados
df_relevamientos_agrupados = pd.DataFrame(agrupados).drop(columns=["agrupado"])

# 8. Para los otros procedimientos, asignar "Cantidad afectada" = 1
df_otros["Cantidad afectada"] = 1

# 9. Unir relevamientos agrupados con los demás procedimientos
df_final = pd.concat([df_relevamientos_agrupados, df_otros], ignore_index=True)

# 10. Exportar archivo final
df_final.to_excel("puntos_con_cantidad_afectada.xlsx", index=False)

print(f"✅ Archivo generado: puntos_con_cantidad_afectada.xlsx (total: {len(df_final)} filas)")


✅ Archivo generado: puntos_con_cantidad_afectada.xlsx (total: 232 filas)


In [10]:
pip install pandas shapely openpyxl


Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd
from shapely.geometry import Point
from shapely.wkt import loads as load_wkt

# --- 1. Definir los polígonos por barrio ---
poligonos = {
    "AMPLIACION VILLA ALEM": load_wkt("POLYGON ((-65.2048 -26.855388, -65.202604 -26.855857, -65.2023 -26.855294, -65.202173 -26.855057, -65.201932 -26.854611, -65.201751 -26.854086, -65.20156 -26.853533, -65.215786 -26.850142, -65.216669 -26.852873, -65.213528 -26.853586, -65.21031 -26.854289, -65.207177 -26.85507, -65.20568 -26.855197, -65.2048 -26.855388))"),
    "EGEO": load_wkt("POLYGON ((-65.214337 -26.857495, -65.211169 -26.858049, -65.210612 -26.855591, -65.213745 -26.854882, -65.214337 -26.857495))"),
    "CAPITAN GIACHINO": load_wkt("POLYGON ((-65.212049 -26.861903, -65.212043 -26.861872, -65.211724 -26.860547, -65.215344 -26.859935, -65.215475 -26.859979, -65.217237 -26.859607, -65.21803 -26.859367, -65.218839 -26.859122, -65.219003 -26.8597, -65.219186 -26.860342, -65.219445 -26.861131, -65.212202 -26.862527, -65.212049 -26.861903))"),
    "VILLA ALEM": load_wkt("POLYGON ((-65.219426 -26.853764, -65.217056 -26.854221, -65.216629 -26.852907, -65.215744 -26.850176, -65.20152 -26.853567, -65.199524 -26.845291, -65.200611 -26.845044, -65.20054 -26.844622, -65.206538 -26.843461, -65.218557 -26.841073, -65.219863 -26.846525, -65.220263 -26.847838, -65.218239 -26.848254, -65.217168 -26.848475, -65.217524 -26.849889, -65.218648 -26.849634, -65.220561 -26.8492, -65.220902 -26.850577, -65.218248 -26.851115, -65.218923 -26.852345, -65.219249 -26.853112, -65.219426 -26.853764))"),
    "VILLA AMALIA": load_wkt("POLYGON ((-65.213929 -26.857564, -65.214326 -26.857495, -65.2179 -26.856701, -65.21822 -26.857499, -65.218257 -26.857593, -65.218476 -26.857543, -65.219709 -26.857282, -65.222421 -26.856813, -65.222587 -26.857124, -65.222734 -26.857542, -65.222024 -26.857617, -65.221478 -26.857825, -65.221119 -26.857961, -65.219955 -26.858454, -65.218828 -26.859122, -65.218427 -26.859243, -65.217664 -26.859475, -65.217226 -26.859607, -65.217121 -26.859222, -65.216616 -26.859286, -65.216377 -26.859289, -65.216043 -26.859291, -65.215596 -26.859184, -65.215257 -26.858942, -65.214826 -26.859036, -65.214696 -26.858529, -65.214165 -26.858656, -65.21378 -26.858749, -65.213736 -26.858544, -65.21368 -26.8583, -65.213663 -26.858212, -65.213654 -26.858169, -65.213601 -26.857927, -65.212794 -26.858045, -65.212861 -26.858403, -65.212918 -26.858705, -65.21296 -26.858927, -65.212183 -26.8591, -65.212205 -26.859239, -65.212252 -26.85953, -65.211861 -26.859632, -65.211538 -26.859716, -65.211713 -26.860547, -65.212032 -26.861872, -65.207085 -26.862843, -65.206345 -26.861629, -65.205726 -26.860678, -65.204879 -26.859377, -65.202593 -26.855857, -65.204372 -26.855477, -65.205669 -26.855197, -65.207166 -26.85507, -65.210299 -26.854289, -65.213517 -26.853586, -65.216658 -26.852873, -65.217084 -26.854187, -65.213734 -26.854882, -65.210601 -26.855591, -65.211158 -26.858049, -65.213929 -26.857564))"),  # abreviado por espacio
    "BLAS VICTORIO CONRERO": load_wkt("POLYGON ((-65.221909 -26.854367, -65.22194 -26.854454, -65.222048 -26.854751, -65.222392 -26.855702, -65.223112 -26.856999, -65.222598 -26.857124, -65.222432 -26.856813, -65.219616 -26.857304, -65.218487 -26.857543, -65.218636 -26.856802, -65.218709 -26.856444, -65.218772 -26.856293, -65.218835 -26.856142, -65.218926 -26.855928, -65.219063 -26.855659, -65.219118 -26.855552, -65.219229 -26.855336, -65.219796 -26.855186, -65.219605 -26.854384, -65.219472 -26.853742, -65.220474 -26.853961, -65.221089 -26.854135, -65.221909 -26.854367))"),
    "SOLDADO GUANCA": load_wkt("POLYGON ((-65.212805 -26.858045, -65.213612 -26.857927, -65.213626 -26.857991, -65.213674 -26.858212, -65.213791 -26.858749, -65.214206 -26.858649, -65.214707 -26.858529, -65.214819 -26.858971, -65.214837 -26.859036, -65.215268 -26.858942, -65.215607 -26.859184, -65.215832 -26.859238, -65.216012 -26.859291, -65.216096 -26.859291, -65.216627 -26.859286, -65.217132 -26.859222, -65.217197 -26.859462, -65.217237 -26.859607, -65.215475 -26.859979, -65.215344 -26.859926, -65.214479 -26.860082, -65.213934 -26.860174, -65.213204 -26.860297, -65.211724 -26.860547, -65.211598 -26.859948, -65.211549 -26.859716, -65.212263 -26.85953, -65.212194 -26.8591, -65.212971 -26.858927, -65.212872 -26.858403, -65.212805 -26.858045))"),
    "76 VIVIENDAS(Ex Refinor)": load_wkt("POLYGON ((-65.218279 -26.84822, -65.219555 -26.847958, -65.220302 -26.847804, -65.2206 -26.849166, -65.218687 -26.8496, -65.217564 -26.849855, -65.217207 -26.848441, -65.218279 -26.84822))"),
    "SARGENTO LAI": load_wkt("POLYGON ((-65.220941 -26.850543, -65.221909 -26.854367, -65.219465 -26.85373, -65.219288 -26.853078, -65.218962 -26.852311, -65.218287 -26.851081, -65.220941 -26.850543))"),
    "JUDICIAL": load_wkt("POLYGON ((-65.219465 -26.853768, -65.219796 -26.855186, -65.219229 -26.855336, -65.219063 -26.855659, -65.218709 -26.856444, -65.218476 -26.857543, -65.218257 -26.857593, -65.2179 -26.856701, -65.214337 -26.857495, -65.213745 -26.854882, -65.219465 -26.853768))"),
    "AMPLIACION VILLA AMALIA": load_wkt("POLYGON ((-65.212202 -26.862527, -65.212434 -26.863658, -65.212586 -26.864298, -65.210545 -26.864751, -65.208911 -26.865079, -65.208596 -26.864543, -65.208352 -26.864629, -65.207096 -26.862843, -65.212043 -26.861872, -65.212202 -26.862527))"),
    "SAN JUSTO": load_wkt("POLYGON ((-65.212434 -26.863658, -65.212202 -26.862527, -65.219461 -26.861129, -65.219872 -26.862285, -65.212434 -26.863658))"),
    "IRIGOYEN (DIZA)": load_wkt("POLYGON ((-65.212586 -26.864298, -65.212434 -26.863658, -65.219872 -26.862285, -65.220308 -26.863603, -65.220997 -26.865688, -65.213234 -26.867181, -65.212822 -26.86535, -65.212586 -26.864298))"),
    "AMPLIACION EL GRAFICO": load_wkt("POLYGON ((-65.220664 -26.865753, -65.220997 -26.865688, -65.221095 -26.866236, -65.221138 -26.866482, -65.220634 -26.866574, -65.218958 -26.866879, -65.218331 -26.866993, -65.217412 -26.86716, -65.216648 -26.867299, -65.215146 -26.867573, -65.213709 -26.867835, -65.213417 -26.867888, -65.213234 -26.867181, -65.214818 -26.866882, -65.216363 -26.866578, -65.217903 -26.866284, -65.220664 -26.865753))"),
    "CAPITAN VIOLA": load_wkt("POLYGON ((-65.212586 -26.864298, -65.212757 -26.865058, -65.213061 -26.866411, -65.213234 -26.867181, -65.213417 -26.867888, -65.21316 -26.867931, -65.211881 -26.868147, -65.210833 -26.868323, -65.210481 -26.867798, -65.210072 -26.86719, -65.209727 -26.866675, -65.209537 -26.866392, -65.208352 -26.864629, -65.208596 -26.864543, -65.208911 -26.865079, -65.212586 -26.864298))"),
    "OLLEROS": load_wkt("POLYGON ((-65.223112 -26.856999, -65.223736 -26.857607, -65.223212 -26.857608, -65.223577 -26.858353, -65.22381 -26.858279, -65.224074 -26.858766, -65.224315 -26.859184, -65.223838 -26.859298, -65.223103 -26.859477, -65.222595 -26.859584, -65.222129 -26.859702, -65.22131 -26.859896, -65.220478 -26.860089, -65.219186 -26.860342, -65.218957 -26.859538, -65.218839 -26.859122, -65.219966 -26.858454, -65.220041 -26.858423, -65.22113 -26.857961, -65.222035 -26.857617, -65.222197 -26.8576, -65.222745 -26.857542, -65.222598 -26.857124, -65.223112 -26.856999))"),
    "SAN NICOLAS": load_wkt("POLYGON ((-65.224455 -26.859492, -65.224597 -26.859774, -65.224997 -26.860571, -65.225461 -26.862634, -65.220308 -26.863603, -65.219872 -26.862285, -65.219186 -26.860342, -65.219866 -26.860208, -65.220478 -26.860089, -65.22131 -26.859896, -65.222723 -26.859557, -65.224303 -26.859188, -65.224455 -26.859492))"),
    "AMPLIACION SAN NICOLAS": load_wkt("POLYGON ((-65.223736 -26.857607, -65.225096 -26.858747, -65.227214 -26.860543, -65.227875 -26.861227, -65.228279 -26.861736, -65.228634 -26.862296, -65.227399 -26.862535, -65.226526 -26.862437, -65.225461 -26.862634, -65.224997 -26.860571, -65.224303 -26.859188, -65.22381 -26.858279, -65.223577 -26.858353, -65.223212 -26.857608, -65.223736 -26.857607))"),
}

# --- 2. Cargar archivo con puntos ---
df = pd.read_excel("puntos_con_cantidad_afectada.xlsx")
df.columns = df.columns.str.strip()
df["Latitud"] = pd.to_numeric(df["Latitud"], errors="coerce")
df["Longitud"] = pd.to_numeric(df["Longitud"], errors="coerce")
df = df.dropna(subset=["Latitud", "Longitud"])

# --- 3. Asignar barrio a cada punto ---
def asignar_barrio(lat, lon):
    punto = Point(lon, lat)
    for nombre_barrio, poligono in poligonos.items():
        if poligono.contains(punto):
            return nombre_barrio
    return "Fuera de los barrios"

df["Barrio"] = df.apply(lambda row: asignar_barrio(row["Latitud"], row["Longitud"]), axis=1)

# --- 4. Exportar resultado ---
df.to_excel("puntos_con_barrios.xlsx", index=False)
print(f"✅ Archivo exportado con {len(df)} puntos y columna 'Barrio' agregada.")


✅ Archivo exportado con 232 puntos y columna 'Barrio' agregada.


In [15]:
import pandas as pd
import re

# 1. Cargar archivo de puntos ya procesado con barrios
df = pd.read_excel("puntos_con_barrios.xlsx")

# 2. Función corregida para extraer solo el nombre de la calle (sin altura ni intersección)
def extraer_calle(direccion):
    direccion = str(direccion).strip()
    match = re.match(r"^(.*?)(\s\d+\b| y\b| y\.| Y\.| Y\b)", direccion)
    if match:
        return match.group(1).strip().title()
    else:
        return direccion.split(",")[0].strip().title()

# 3. Función para extraer número o calle de la intersección
def extraer_interseccion(direccion):
    direccion = str(direccion).strip()
    if " y " in direccion.lower():
        parte = direccion.lower().split(" y ")[-1].split(",")[0].strip()
        return parte.split()[0].title() if parte else ""
    elif " y." in direccion.lower():
        parte = direccion.lower().split(" y. ")[-1].split(",")[0].strip()
        return parte.split()[0].title() if parte else ""
    elif " y" in direccion.lower():
        parte = direccion.lower().split(" y")[-1].split(",")[0].strip()
        return parte.split()[0].title() if parte else ""
    else:
        partes = direccion.split(",")[0].split()
        if partes:
            ultima = partes[-1]
            return ultima if ultima.isdigit() else ultima.title()
        return ""

# 4. Aplicar ambas funciones
df["Direccion_Calle"] = df["Direccion"].apply(extraer_calle)
df["Direccion_Numero_Interseccion"] = df["Direccion"].apply(extraer_interseccion)

# 5. Crear DataFrame final
df_final = pd.DataFrame({
    "ID_Punto": "",
    "Tipo_Intervencion_ID": "18",
    "Subtipo_Intervencion_ID": "todos",
    "Descripcion_Detallada": df["Procedimiento Destacado"],
    "Latitud": df["Latitud"],
    "Longitud": df["Longitud"],
    "Direccion_Calle": df["Direccion_Calle"],
    "Direccion_Numero_Interseccion": df["Direccion_Numero_Interseccion"],
    "Barrio_ID": df["Barrio"],
    "Distrito_ID": "15",
    "Fecha_Realizacion": df["Fecha"],
    "Cantidad_Afectada": df["Cantidad afectada"],
    "Unidad_Medida": "locales gastronómicos",
    "Estado_Actual": "completados",
    "Responsable_Direccion_ID": "18",
    "Notas_Adicionales": ""
})

# 6. Exportar a Excel
df_final.to_excel("intervenciones_formato_final.xlsx", index=False)
print("✅ Archivo final exportado: 'intervenciones_formato_final.xlsx'")



✅ Archivo final exportado: 'intervenciones_formato_final.xlsx'


In [16]:
pip install folium pandas




   ---------------- ----------------------- 2/5 [jinja2]
   ---------------- ----------------------- 2/5 [jinja2]
   ---------------- ----------------------- 2/5 [jinja2]
   ---------------- ----------------------- 2/5 [jinja2]
   ------------------------ --------------- 3/5 [branca]
   -------------------------------- ------- 4/5 [folium]
   -------------------------------- ------- 4/5 [folium]
   -------------------------------- ------- 4/5 [folium]
   -------------------------------- ------- 4/5 [folium]
   -------------------------------- ------- 4/5 [folium]
   -------------------------------- ------- 4/5 [folium]
   ---------------------------------------- 5/5 [folium]



In [18]:
import pandas as pd
import folium

# 1. Cargar el archivo con datos ya procesados
df = pd.read_excel("intervenciones_formato_final.xlsx")

# 2. Crear el mapa centrado en San Miguel de Tucumán
mapa = folium.Map(location=[-26.833, -65.216], zoom_start=13)

# 3. Agregar cada punto como marcador circular
for _, row in df.iterrows():
    direccion = row.get("Direccion_Calle", "Sin dirección")
    barrio = row.get("Barrio_ID", "Sin barrio")

    folium.CircleMarker(
        location=[row["Latitud"], row["Longitud"]],
        radius=5,
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.6,
        popup=folium.Popup(f"{direccion}<br><b>Barrio:</b> {barrio}", max_width=300)
    ).add_to(mapa)

# 4. Guardar el mapa en un archivo HTML
mapa.save("mapa_inspecciones.html")
print("✅ Mapa exportado como 'mapa_inspecciones.html'")


✅ Mapa exportado como 'mapa_inspecciones.html'


In [19]:
pip install shapely folium pandas


Note: you may need to restart the kernel to use updated packages.


In [20]:
import pandas as pd
import folium
from shapely.wkt import loads as load_wkt
from shapely.geometry import mapping

# 1. Leer puntos desde archivo
df = pd.read_excel("intervenciones_formato_final.xlsx")

# 2. Crear mapa centrado en San Miguel de Tucumán
mapa = folium.Map(location=[-26.833, -65.216], zoom_start=13)

# 3. Agregar puntos
for _, row in df.iterrows():
    direccion = row.get("Direccion_Calle", "Sin dirección")
    barrio = row.get("Barrio_ID", "Sin barrio")

    folium.CircleMarker(
        location=[row["Latitud"], row["Longitud"]],
        radius=5,
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.6,
        popup=folium.Popup(f"{direccion}<br><b>Barrio:</b> {barrio}", max_width=300)
    ).add_to(mapa)

# 4. Agregar polígonos de barrios
for nombre, poligono in poligonos.items():
    coords = [[lat, lon] for lon, lat in poligono.exterior.coords]
    folium.Polygon(
        locations=coords,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.1,
        weight=2,
        popup=f"{nombre}"
    ).add_to(mapa)

# 5. Guardar como archivo HTML
mapa.save("mapa_con_poligonos.html")
print("✅ Mapa generado como 'mapa_con_poligonos.html'")


✅ Mapa generado como 'mapa_con_poligonos.html'
